In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

# Preparing train data

In [2]:
df = pd.read_csv('tag.csv',encoding_errors='ignore')

In [3]:
rooms = []
for i in range(len(df)):
    item_folder = "./train/" + df.loc[i]['item'] + "/"
    natural_bool=df.loc[i]['natural']
    rooms.append((natural_bool, item_folder))

In [4]:
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_folder'])
print(train_df.head())
print(train_df.tail())

         tag           video_folder
0    natural          ./train/bear/
1    natural  ./train/bike-packing/
2    natural     ./train/blackswan/
3  unnatural     ./train/bmx-bumps/
4    natural     ./train/bmx-trees/
          tag           video_folder
84  unnatural         ./train/train/
85  unnatural       ./train/tuk-tuk/
86  unnatural   ./train/upside-down/
87  unnatural  ./train/varanus-cage/
88  unnatural       ./train/walking/


# Preparing test data

In [5]:
df = pd.read_csv('test_tag.csv',encoding_errors='ignore')

In [6]:
rooms = []
for i in range(len(df)):
    item_folder = "./test/" + df.loc[i]['item'] + "/"
    natural_bool=df.loc[i]['natural']
    rooms.append((natural_bool, item_folder))

In [7]:
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_folder'])
print(test_df.head())

         tag            video_folder
0  unnatural             ./test/bus/
1    natural           ./test/camel/
2    natural  ./test/car-roundabout/
3    natural      ./test/car-shadow/


In [8]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

# Data prepartion

In [10]:
print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

Total videos for training: 89
Total videos for testing: 4


# feed the videos to a network

In [11]:
import natsort
import glob
# make frame array
# path = "./train/blackswan/"
def load_frame(path):
    if os.path.isdir(path):
        images = natsort.natsorted(glob.glob(os.path.join(path,"*jpg")))
        images_read = [cv2.imread(frame) for frame in images]
#         print(images_read[0].shape)
#         print(np.array(images_read))
        return np.array(images_read)
# load_frame(path)

## feature extraction

In [12]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(480, 641, 3), #choose later
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((480, 641, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()    
    

## label encoding

In [13]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))

In [14]:
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['natural', 'unnatural']


array([[0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [15]:
# define hyperparameters

IMG_WIDTH = 641
IMG_HEIGHT = 480
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 200
NUM_FEATURES = 2048

In [16]:
train_df

,tag,video_folder
0,natural,./train/bear/
1,natural,./train/bike-packing/
2,natural,./train/blackswan/
3,unnatural,./train/bmx-bumps/
4,natural,./train/bmx-trees/
...,...,...
84,unnatural,./train/train/
85,unnatural,./train/tuk-tuk/
86,unnatural,./train/upside-down/
87,unnatural,./train/varanus-cage/


In [17]:
import pdb
def prepare_all_videos(df):
    num_samples = len(df)
    video_paths = df['video_folder'].tolist()
    labels = df["tag"].values
    
    labels = label_processor(labels[..., None]).numpy()
    
    #storing
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048
    
    for idx, path in enumerate(video_paths):
        print(path)
        frames = load_frame(path)
        frames = frames [None, ...] #??
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            print(length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()
    return (frame_features, frame_masks), labels



In [18]:
import time
start = time.time()
train_data, train_labels = prepare_all_videos(train_df)
test_data, test_labels = prepare_all_videos(test_df)

# test_data, test_labels = prepare_all_videos(test_df, "test")
print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")
print("time :", time.time() - start)

# print(f"test_labels in train set: {test_labels.shape}")


./train/bear/
82
1/1 [==============================] - 0s 125ms/step
./train/bike-packing/
69
1/1 [==============================] - 0s 136ms/step
./train/blackswan/
50
1/1 [==============================] - 0s 160ms/step


1/1 [==============================] - 0s 130ms/step
./train/bmx-bumps/
90
1/1 [==============================] - 0s 130ms/step
./train/bmx-trees/
80
1/1 [==============================] - 0s 135ms/step


1/1 [==============================] - 0s 131ms/step
./train/boat/
73
1/1 [==============================] - 0s 128ms/step
./train/boxing-fisheye/
65
1/1 [==============================] - 0s 128ms/step


1/1 [==============================] - 0s 136ms/step
./train/breakdance/
43
1/1 [==============================] - 0s 137ms/step
./train/breakdance-flare/
70
1/1 [==============================] - 0s 134ms/step


1/1 [==============================] - 0s 133ms/step
./train/bus/
64
1/1 [==============================] - 0s 132ms/step
./train/camel/
43
1/1 [==============================] - 0s 137ms/step
./train/car-roundabout/
40
1/1 [==============================] - 0s 133ms/step


...

1/1 [==============================] - 0s 343ms/step
./train/surf/
35
1/1 [==============================] - 0s 374ms/step
./train/swing/
60
1/1 [==============================] - 0s 341ms/step
./train/tennis/
50
1/1 [==============================] - 0s 334ms/step


1/1 [==============================] - 0s 331ms/step
./train/tractor-sand/
76
1/1 [==============================] - 0s 309ms/step
./train/train/
80
1/1 [==============================] - 0s 335ms/step


1/1 [==============================] - 0s 328ms/step
./train/tuk-tuk/
59
1/1 [==============================] - 0s 341ms/step
./train/upside-down/
65
1/1 [==============================] - 0s 361ms/step
./train/varanus-cage/
49
1/1 [==============================] - 0s 360ms/step


1/1 [==============================] - 0s 362ms/step
./train/walking/
40
1/1 [==============================] - 0s 366ms/step
./test/bus/
64
1/1 [==============================] - 0s 380ms/step
./test/camel/
43
1/1 [==============================] - 0s 373ms/step


1/1 [==============================] - 0s 385ms/step
./test/car-roundabout/
40
1/1 [==============================] - 0s 361ms/step
./test/car-shadow/
40
1/1 [==============================] - 0s 399ms/step
Frame features in train set: (89, 200, 2048)
Frame masks in train set: (89, 200)
train_labels in train set: (89, 1)
test_labels in train set: (4, 1)
time : 1276.134164571762


# The sequence model

In [22]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./checkpoint/"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    pdb.set_trace()
    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
2/2 [==============================] - ETA: 0s - loss: 0.7210 - accuracy: 0.4677
Epoch 1: val_loss improved from inf to 0.63834, saving model to ./checkpoint\
2/2 [==============================] - 7s 2s/step - loss: 0.7210 - accuracy: 0.4677 - val_loss: 0.6383 - val_accuracy: 0.6667
Epoch 2/30
2/2 [==============================] - ETA: 0s - loss: 0.6683 - accuracy: 0.6290
Epoch 2: val_loss improved from 0.63834 to 0.62439, saving model to ./checkpoint\
2/2 [==============================] - 1s 373ms/step - loss: 0.6683 - accuracy: 0.6290 - val_loss: 0.6244 - val_accuracy: 0.6667
Epoch 3/30
2/2 [==============================] - ETA: 0s - loss: 0.6714 - accuracy: 0.6129
Epoch 3: val_loss improved from 0.62439 to 0.62326, saving model to ./checkpoint\
2/2 [==============================] - 1s 364ms/step - loss: 0.6714 - accuracy: 0.6129 - val_loss: 0.6233 - val_accuracy: 0.6667
Epoch 4/30
2/2 [==============================] - ETA: 0s - loss: 0.6482 - accuracy: 0.6129
Epoch 

Epoch 29/30
2/2 [==============================] - ETA: 0s - loss: 0.3368 - accuracy: 0.9355
Epoch 29: val_loss improved from 0.52863 to 0.52304, saving model to ./checkpoint\
2/2 [==============================] - 1s 368ms/step - loss: 0.3368 - accuracy: 0.9355 - val_loss: 0.5230 - val_accuracy: 0.7407
Epoch 30/30
2/2 [==============================] - ETA: 0s - loss: 0.2618 - accuracy: 1.0000
Epoch 30: val_loss did not improve from 0.52304
1/1 [==============================] - 0s 45ms/step - loss: 0.2945 - accuracy: 1.0000
Test accuracy: 100.0%
> c:\users\vml_sub\appdata\local\temp\ipykernel_25372\3109293825.py(44)run_experiment()

ipdb> accuracy
1.0
ipdb> test_data[0]
array([[[0.28409937, 0.71432793, 0.35459766, ..., 0.4129823 ,
         0.2285323 , 0.56644136],
        [0.33988518, 0.7563973 , 0.33714885, ..., 0.42955813,
         0.20257095, 0.5605487 ],
        [0.3118559 , 0.7525436 , 0.35853204, ..., 0.33480424,
         0.19573018, 0.64574033],
        ...,
        [0.       

ipdb> test_labels
array([[1],
       [0],
       [0],
       [0]], dtype=int64)
ipdb> seq_model.evaluate([test_data[0], test_data[1]], test_labels)
1/1 [==============================] - 0s 55ms/step - loss: 0.2945 - accuracy: 1.0000
[0.2944643497467041, 1.0]
ipdb> exit


BdbQuit: 

# Inference

In [21]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_frameameameameamerameramerame(path)
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = "./test/camel"
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: ./test/camel


NameError: name 'load_frameameameameamerameramerame' is not defined

In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="dataset/test/dancing/dancing (23.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")
